# Calculating the Levelized Cost of Charging
This notebook is based on lcoc_demo.ipynb but has been modified to calculate county-level LCOC, including county-level electricity prices and fuel efficiency (if provided, the default data used here is from Borlaug et al.). The functions have also been modified to included multiple vehicle classes (if provided). Inclusion of these factors is only applicable to residential LCOC and depends on the data fed into the `calculate_county_residential_lcoc` function.


Original text from Borlaug et al. version:

EIA reports and [The Utility Rate Database](https://openei.org/wiki/Utility_Rate_Database) were used to estimate the annual cost of electricity for EV charging loads at the rate-, utility-, and state-levels. For residential TOU rates (URDB), it is assumed that the consumer will always charge at the lowest possible rate (off-peak). In determining the cost of electricity for residential charging, rates with demand charges are not included as they would require typical household energy profiles that vary greatly on a case-by-case basis. For estimating the cost of electricity for commercial use (e.g. DCFC stations), typical energy profiles (electricity demand scenarios) were generated to estimate fixed, demand, and energy charges for a variety of rate structures (TOU, tier, seasonal, demand, etc.).

In [1]:
import sys
import logging
import numpy as np
import pandas as pd

sys.path.append(os.path.join('..',''))
import config
import lcoc.urdb as urdb
import lcoc.afdc as afdc
import lcoc.processing as proc
import os
from IPython.display import display
import xlwings as xw

## Set parameters

In [2]:
# Specify version of URDB and AFDC datasets to use. If a specific date, this must be specified in the config file. The current setting will download today's version of the datasets.
urdb_afdc_version = 'specific_date' # 'current'

# Determine whether to run DCFC cost of electricity (COE) calculations, which include the EVI-FAST analysis that takes 3.5-4 hours. Only necessary to re-run if using new data or assumptions that deviate from the original analysis.
# If True, DCFC COE analysis will run
# If False, the results from the previous run will be used
run_DCFC_COE = False

## Load Data

### EIA Data Processing
**Data Source Information**  
Note that a full census of all utilities is only collected every 8 years, the most recent of which was in 2019. The 2019 data is updated with 2020 data for the sample collected in 2020.
 - Annual Energy Outlook (eia_aeo_raw) data downloaded from https://www.eia.gov/outlooks/aeo/data/browser/#/?id=3-AEO2021&region=1-0&cases=ref2021~highprice~lowprice&start=2020&end=2050&f=A&linechart=~~~ref2021-d113020a.28-3-AEO2021.1-0~highprice-d113020a.28-3-AEO2021.1-0~lowprice-d113020a.28-3-AEO2021.1-0~ref2021-d113020a.13-3-AEO2021.1-0~highprice-d113020a.13-3-AEO2021.1-0~lowprice-d113020a.13-3-AEO2021.1-0~ref2021-d113020a.6-3-AEO2021.1-0~highprice-d113020a.6-3-AEO2021.1-0~lowprice-d113020a.6-3-AEO2021.1-0&map=highprice-d113020a.3-3-AEO2021.1-0&chartindexed=1&sourcekey=0
     - If using a different AEO year, will need to recreate the chart for the desired AEO year and download data
 - table6 & table7 price files downloaded from https://www.eia.gov/electricity/sales_revenue_price/, need to be renamed with _YY for clarity
 - Service_Territory_YYYY & Delivery_Companies_YYYY downloaded from https://www.eia.gov/electricity/data/eia861/. 
     - Download zip file, unzip
     - Copy Service_Territory_YYYY to eia/eiaid-crosswalk
     - Copy Delivery_Companies_YYYY to eia/TX-delivery-companies
 - Run code chunk below

In [3]:
## Set import parameters
eia_price_yr_c = '2019' # c = census
eia_price_yr_s = '2020' # s = sample
aeo_yr = '2021'

fips_state_county = proc.state_county_FIPS(fips_state_file_path=os.path.join(config.DATA_PATH,'census_bureau','fips_state.txt'),
                                            fips_state_county_file_path=os.path.join(config.DATA_PATH,'census_bureau','national_county.txt'))
proc.eia_data(eia_price_yr_c=eia_price_yr_c,eia_price_yr_s=eia_price_yr_s,aeo_yr=aeo_yr,fips_state_county=fips_state_county)
eia_aeo = pd.read_csv(os.path.join(config.DATA_PATH,'eia','15yr-gas-electricity-price-projections',f'eia_price_scenarios_aeo{aeo_yr[-2:]}.csv'))

Loading data.
Data loaded.
AEO data processed.
2019 census EIA residential electricity price data contains 3048 utility-state groups, 2020 sample EIA residential electricity price data contains 1439 utility-state groups
2019 census EIA commercial electricity price data contains 3043 utility-state groups, 2020 census EIA commercial electricity price data contains 1453 utility-state groups
Updating entities in Census price data with most recent Service Territory list
Number of residential utilities in sample but not in census data: 0
Number of commercial utilities in sample but not in census data: 0
Updating census data with sample prices where applicable
Reconciling Texas Retail Energy Providers (REP) / Retail Power Marketer (RPM) and TDU mismatch
Customer-weighted average residential price for Texas RPMs is 12.65 cents per kWh
Customer-weighted average commercial price for Texas RPMs is 9.35 cents per kWh
Final residential electricity price data contains 2980 utility-state groups
Final

### Load & Process Data

In [4]:
## LOAD DATA ##
if urdb_afdc_version == 'current':
    db = urdb.DatabaseRates() # gets current snapshot of URDB and saves it to db
elif urdb_afdc_version == 'specific_date':
    urdb_path = config.URDB_PATH
    print(config.URDB_PATH)
    db = urdb.DatabaseRates(urdb_path)

print("Total:", db.rate_data.shape)
print("Res:", db.res_rate_data.shape)
print("Com:", db.com_rate_data.shape)

print('filter expired rates')
db.filter_stale_rates(industry='residential')
print("Res:", db.res_rate_data.shape)

db.filter_stale_rates(industry='commercial')
print("Com:", db.com_rate_data.shape)

# update commercial rates with manual changes to demand and energy usage limits (included in description but missing from database fields)
urdb_rates_update = pd.read_excel(os.path.join(config.DATA_PATH,'urdb','urdb_rates_update_202205.xlsx'),sheet_name='changed_rates')
urdb_rates_update.set_index(keys='label',inplace=True)
db.com_rate_data.set_index(keys='label',inplace=True)
db.com_rate_data.update(urdb_rates_update,overwrite=True)
db.com_rate_data = db.com_rate_data.reset_index()

# classify rates by is_tier, is_seasonal, is_TOU, is_ev-specific (residential only)
ev_rate_words_filepath = os.path.join(config.HOME_PATH,'filters','urdb_res_ev_specific_rate_words.txt')

db.classify_rate_structures(industry='residential', 
                            ev_rate_words_file=ev_rate_words_filepath)

db.classify_rate_structures(industry='commercial')

# standardize units of reporting for commercial rates
db.com_rate_preprocessing()

filters_path = os.path.join(config.HOME_PATH,'filters')

# filter demand rates (residential only)
db.filter_demand_rates(industry='residential') 

# filter commercial rates missing critical fields to approx the cost of electricity
db.additional_com_rate_filters()

# filter rates containing certain phrases in filters/
db.filter_on_phrases(industry='residential', filters_path=filters_path)
db.filter_on_phrases(industry='commercial', filters_path=filters_path)

# combine base rate + adjusted rate
db.combine_rates(industry='residential')
db.combine_rates(industry='commercial')

# filter null rates
db.filter_null_rates(industry='residential')
db.filter_null_rates(industry='commercial')

print('size of rate dataframes:')
print("Res:", db.res_rate_data.shape)
print("Com:", db.com_rate_data.shape)

print('res rate structure breakdown')
db.generate_classification_tree_values(industry='residential')

print('com rate structure breakdown')
db.generate_classification_tree_values(industry='commercial')

C:\Users\Jesse Vega-Perkins\Documents\thesis_ev\02_analysis\lcoc-ldevs\data\urdb\usurdb_20220526.csv
Total: (50311, 670)
Res: (11460, 670)
Com: (32503, 670)
filter expired rates
Res: (6151, 670)
Com: (17961, 670)
size of rate dataframes:
Res: (4772, 732)
Com: (7541, 731)
res rate structure breakdown
com rate structure breakdown


{'demand': 4196,
 'no_demand': 3345,
 'demand/tier': 1282,
 'demand/fixed': 2914,
 'no_demand/tier': 1299,
 'no_demand/fixed': 2046,
 'demand/tier/seasonal': 158,
 'demand/tier/no_seasonal': 1124,
 'demand/fixed/seasonal': 517,
 'demand/fixed/no_seasonal': 2397,
 'no_demand/tier/seasonal': 326,
 'no_demand/tier/no_seasonal': 973,
 'no_demand/fixed/seasonal': 320,
 'no_demand/fixed/no_seasonal': 1726,
 'demand/tier/seasonal/tou': 21,
 'demand/tier/seasonal/no_tou': 137,
 'demand/tier/no_seasonal/tou': 13,
 'demand/tier/no_seasonal/no_tou': 1111,
 'demand/fixed/seasonal/tou': 240,
 'demand/fixed/seasonal/no_tou': 277,
 'demand/fixed/no_seasonal/tou': 243,
 'demand/fixed/no_seasonal/no_tou': 2154,
 'no_demand/tier/seasonal/tou': 7,
 'no_demand/tier/seasonal/no_tou': 319,
 'no_demand/tier/no_seasonal/tou': 4,
 'no_demand/tier/no_seasonal/no_tou': 969,
 'no_demand/fixed/seasonal/tou': 96,
 'no_demand/fixed/seasonal/no_tou': 224,
 'no_demand/fixed/no_seasonal/tou': 171,
 'no_demand/fixed/no_

### Residential & Workplace/Public-L2 COE & LCOC

In [5]:
## RESIDENTIAL COE and LCOC ##
print('calculate annual electricity cost (rates)')
db.calculate_annual_energy_cost_residential_v2(outpath = os.path.join(config.OUTPUT_PATH,'cost-of-electricity','urdb-res-rates'))

print('calculate annual electricity cost (utility-level)')
df = proc.res_rates_to_utils(
    scenario='baseline',
    urdb_rates_file=os.path.join(config.OUTPUT_PATH,'cost-of-electricity','urdb-res-rates','res_rates.csv'),
    eia_cw_file=config.EIAID_TO_UTILITY_CW_PATH,
    eia_utils_file=config.EIA_RES_PATH,
    outpath=os.path.join(config.OUTPUT_PATH,'cost-of-electricity','res-utilities'))

print('calculate annual electricity cost (county-level)')
proc.res_utils_to_county_and_state(
    utils_file=os.path.join(config.OUTPUT_PATH,'cost-of-electricity','res-utilities','res_utils.csv'), #lower_bnd_res_utils.csv, upper_bnd_res_utils.csv,
    util_county_cw_file=config.EIAID_TO_COUNTY_CW_PATH,
    outfile_county=os.path.join(config.OUTPUT_PATH,'cost-of-electricity','res-counties','res_counties_baseline.csv'),
    outfile_state=os.path.join(config.OUTPUT_PATH,'cost-of-electricity','res-states','res_states_baseline.csv')) #res_states_lower_bnd.csv, res_states_upper_bnd.csv
print('calculate levelized cost of charging (county-level)')
lcoc_res_df = proc.calculate_county_residential_lcoc(
    coe_file = os.path.join(config.OUTPUT_PATH,'cost-of-electricity','res-counties','res_counties_baseline.csv'),
    county_fc_df=pd.read_csv(os.path.join(config.HOME_PATH,'outputs','veh_param','county_fuel_consumption.csv')), # fuel consumption by county (default sedan from Borlaug et al. used here)
    lifetime_vmt_df=pd.read_csv(os.path.join(config.HOME_PATH,'outputs','veh_param','lifetime_vmt.csv')), # lifetime VMT by vehicle class (default sedan from Borlaug et al. used here)
    fixed_costs_path = os.path.join(config.DATA_PATH,'fixed-costs','residential'),
    fixed_costs_filenames = ['res_level1.txt', 'res_level2.txt'],
    annual_maint_frac = 0.02, # Annual cost of maintenance (fraction of equip costs)
    equip_lifespan = 10, # source: https://afdc.energy.gov/files/u/publication/evse_cost_report_2015.pdf
    charge_eff = 0.85,
    fraction_residential_charging = 0.81, 
    fraction_home_l1_charging = 0.16, 
    dr = 0.035, 
    print_message = False,
    outfile = os.path.join(config.OUTPUT_PATH,'cost-of-charging','residential','res_counties_baseline.csv'))


## PUBLIC COE and LCOC ##
print('calculate annual electricity cost (county-level)')
proc.com_utils_to_county(utils_file = config.EIA_COM_PATH,
                        util_county_cw_file = config.EIAID_TO_COUNTY_CW_PATH,
                        outfile_county = os.path.join(config.OUTPUT_PATH,'cost-of-electricity','com-counties','com_counties_baseline.csv'),
                        outfile_state = os.path.join(config.OUTPUT_PATH,'cost-of-electricity','com-states','com_states_baseline.csv'))
print('calculate levelized cost of charging (county-level)')
proc.calculate_county_workplace_public_l2_lcoc(coe_path = os.path.join(config.OUTPUT_PATH,'cost-of-electricity','com-counties','com_counties_baseline.csv'),
                                    fixed_costs_file = os.path.join(config.DATA_PATH,'fixed-costs','workplace-public-l2','com_level2.txt'),
                                    annual_maint_frac = 0.08, # source: NREL communication
                                    equip_lifespan = 10, # source: https://afdc.energy.gov/files/u/publication/evse_cost_report_2015.pdf
                                    equip_utilization_kwh_per_day = 30, # source: INL
                                    outpath = os.path.join(config.OUTPUT_PATH,'cost-of-charging','workplace-public-l2','work_pub_l2_counties_baseline.csv'))

calculate annual electricity cost (rates)
Complete, 4766 rates included.
calculate annual electricity cost (utility-level)
Complete, 2980 utitilies represented (308 TOU rates used).
calculate annual electricity cost (county-level)
County aggregation complete, 3142 counties represented
State aggregation complete, 51 states (including national average [US]) represented
National weighted average complete, national cost of electricity is $0.1/kWh.
calculate levelized cost of charging (county-level)
calculate annual electricity cost (county-level)
County aggregation complete, 3142 counties represented
State aggregation complete, 51 states (including national average [US]) represented
National weighted average complete, national commercial cost of electricity is $0.14/kWh.
calculate levelized cost of charging (county-level)
County-level LCOC calculation complete, national LCOC (workplace/pub-L2) is $0.24/kWh


### DCFC COE
This block only runs if `run_DCFC_COE` is set equal to `True` at the top of this notebook. Otherwise, this block is skipped b/c it takes 3.5-4 hours to run.

In [6]:
if run_DCFC_COE == True:
    db.calculate_annual_cost_dcfc_v2(dcfc_load_profiles = config.DCFC_PROFILES_DICT,
                                    outpath=os.path.join(config.OUTPUT_PATH,'cost-of-electricity','urdb-dcfc-rates'),
                                    log_lvl=1) #0=WARNING, 1=INFO, 2=DEBUG
    # calculate annual electricity cost (utility-level)
    proc.dcfc_rates_to_utils(urdb_rates_files = config.DCFC_PROFILES_DICT,
                            inpath = os.path.join(config.OUTPUT_PATH,'cost-of-electricity','urdb-dcfc-rates'),
                            outpath = os.path.join(config.OUTPUT_PATH,'cost-of-electricity','urdb-dcfc-utilities'))
    #AFDC
    if urdb_afdc_version == 'current':
        stations = afdc.DCFastChargingLocator() #GET DCFC stations from AFDC
    elif urdb_afdc_version == 'specific_date':
        stations = afdc.DCFastChargingLocator(config.AFDC_PATH) #GET DCFC stations from AFDC
    stations.join_county_geoid(us_counties_gdf_file = os.path.join(config.DATA_PATH,'gis','2019_counties','cb_2019_us_county_500k','cb_2019_us_county_500k.shp')) #join to county (spatial)
    stations.aggregate_counties_to_csv(outfile = os.path.join(config.OUTPUT_PATH,'county-dcfc-counts','afdc_county_station_counts.csv')) #aggregate to county-lvl, output to .csv

    # calculate annual electricity cost (county and state-level)
    proc.dcfc_utils_to_county_and_state(
        urdb_util_files = { 'p1':os.path.join(config.OUTPUT_PATH,'cost-of-electricity','urdb-dcfc-utilities','dcfc_utils_p1.csv'),
                            'p2':os.path.join(config.OUTPUT_PATH,'cost-of-electricity','urdb-dcfc-utilities','dcfc_utils_p2.csv'),
                            'p3':os.path.join(config.OUTPUT_PATH,'cost-of-electricity','urdb-dcfc-utilities','dcfc_utils_p3.csv'),
                            'p4':os.path.join(config.OUTPUT_PATH,'cost-of-electricity','urdb-dcfc-utilities','dcfc_utils_p4.csv')},
        eia_territory_file = config.EIAID_TO_COUNTY_CW_PATH,
        outpath_county = os.path.join(config.OUTPUT_PATH,'cost-of-electricity','urdb-dcfc-counties'),
        afdc_counties_file = os.path.join(config.OUTPUT_PATH,'county-dcfc-counts','afdc_county_station_counts.csv'),
        outpath_state = os.path.join(config.OUTPUT_PATH,'cost-of-electricity','dcfc-states'))
    
    # EVI-FAST
    # WARNING: This step takes about 230 min. (~4 hours)
    file_macrobook = os.path.join(config.HOME_PATH,'evi-fast-tool','lcoc_county_dcfc.xlsm')
    file_evi_fast = os.path.join(config.HOME_PATH,'evi-fast-tool','evi-fast_dcfc_lcoc.xlsm')
    dcfc_scenarios = {
                        1:'baseline_dcfc_station_p1_lcoc',
                        2:'min_dcfc_station_p1_lcoc',
                        3:'max_dcfc_station_p1_lcoc',
                        4:'baseline_dcfc_station_p2_lcoc',
                        5:'min_dcfc_station_p2_lcoc',
                        6:'max_dcfc_station_p2_lcoc',
                        7:'baseline_dcfc_station_p3_lcoc',
                        8:'min_dcfc_station_p3_lcoc',
                        9:'max_dcfc_station_p3_lcoc',
                        10:'baseline_dcfc_station_p4_lcoc',
                        11:'min_dcfc_station_p4_lcoc',
                        12:'max_dcfc_station_p4_lcoc'
                        }
    proc.dcfc_county_to_lcoc_evifast(file_macrobook=file_macrobook,file_evi_fast=file_evi_fast,dcfc_scenarios=dcfc_scenarios)
    proc.dcfc_county_lcoc_results_proc(file_macrobook=file_macrobook,outpath=os.path.join(config.OUTPUT_PATH,'cost-of-charging','dcfc'))

# produce baseline, min, and max DCFC LCOC from 4 profiles
# baseline
print('baseline:')
dcfc_lcoc_file = os.path.join(config.OUTPUT_PATH,'cost-of-charging','dcfc','dcfc_counties_baseline.csv')
proc.combine_county_dcfc_profiles_into_single_lcoc(dcfc_lcoc_file = dcfc_lcoc_file,
                                        load_profile_path = config.DCFC_PROFILES_DICT,
                                        afdc_path = config.AFDC_PATH)
# min
print('min:')
dcfc_lcoc_file = os.path.join(config.OUTPUT_PATH,'cost-of-charging','dcfc','dcfc_counties_min.csv')
proc.combine_county_dcfc_profiles_into_single_lcoc(dcfc_lcoc_file = dcfc_lcoc_file,
                                        load_profile_path = config.DCFC_PROFILES_DICT,
                                        afdc_path = config.AFDC_PATH)
# max
print('max:')
dcfc_lcoc_file = os.path.join(config.OUTPUT_PATH,'cost-of-charging','dcfc','dcfc_counties_max.csv')
proc.combine_county_dcfc_profiles_into_single_lcoc(dcfc_lcoc_file = dcfc_lcoc_file,
                                        load_profile_path = config.DCFC_PROFILES_DICT,
                                        afdc_path = config.AFDC_PATH)

baseline:
LCOC calculation complete, national LCOC (DCFC) is $0.25/kWh
min:
LCOC calculation complete, national LCOC (DCFC) is $0.2/kWh
max:
LCOC calculation complete, national LCOC (DCFC) is $0.31/kWh


## Combined LCOC

In [7]:
# Combine
lcoc_df = proc.combine_county_res_work_dcfc_lcoc(
                    res_df = pd.read_csv(os.path.join(config.HOME_PATH,'outputs','cost-of-charging','residential','res_counties_baseline.csv')),
                    wrk_df = pd.read_csv(os.path.join(config.HOME_PATH,'outputs','cost-of-charging','workplace-public-l2','work_pub_l2_counties_baseline.csv')),
                    dcfc_df = pd.read_csv(os.path.join(config.HOME_PATH,'outputs','cost-of-charging','dcfc','dcfc_counties_baseline.csv')),
                    eia_county_lookup_df = pd.read_csv(config.EIA_TO_COUNTY_LOOKUP_PATH),
                    res_wgt = 0.81,
                    wrk_wgt = 0.14,
                    dcfc_wgt = 0.05,
                    print_message = True,
                    outpath = os.path.join('..','outputs','cost-of-charging','comb'))

Combined county-level LCOC calculation complete, national LCOC is $0.16/kWh
